In [1]:
import pandas as pd
from pathlib import Path
ROOT = Path('../..')
ROOT.resolve()

PosixPath('/Users/lukestrange/Code/housing')

In [2]:
dwelling_stock = pd.read_excel(ROOT / 'raw/LT_125.ods', engine='odf', sheet_name='Table_125_(unrounded)', header=3)
dwelling_stock.drop(columns=['Old ONS code'], inplace=True)
mapper = {}
for col in dwelling_stock.columns.to_list()[2:]:
    if len(col) > 4:
        mapper[col] = col[0:4]
dwelling_stock.rename(columns=mapper, inplace=True)
dates = dwelling_stock.columns.to_list()[2:]
dwelling_stock = dwelling_stock.melt(id_vars=['New ONS code', 'Area'], var_name='date', value_name='Value', value_vars=dates)
dwelling_stock['Measure'] = 'Dwellings'
dwelling_stock.rename(columns={'New ONS code': 'geography_code', 'Area': 'geography_name'}, inplace=True)
dwelling_stock.replace({'NORTH EAST': 'North East', 
                        'NORTH WEST': 'North West', 
                        'YORKSHIRE AND THE HUMBER': 'Yorkshire and The Humber', 
                        'EAST MIDLANDS': 'East Midlands', 
                        'WEST MIDLANDS': 'West Midlands', 
                        'EAST OF ENGLAND': 'East of England', 
                        'LONDON': 'London', 
                        'SOUTH EAST': 'South East',
                        'SOUTH WEST': 'South West',
                        'ENGLAND': 'England',
                        'E11000004': 'E11000007',
                        'St. Albans': 'St Albans',
                        'St Helens': 'St. Helens',
                        'Folkestone and Hythe [note 7]': 'Folkestone and Hythe',
                        'St. Edmundsbury': 'St Edmundsbury',
                        r"&": 'and'}, inplace=True)
dwelling_stock = dwelling_stock.loc[dwelling_stock['date'].astype(int) >= 2004]
dwelling_stock

,geography_code,geography_name,date,Value,Measure
1269,E92000001,England,2004,21684360.255307,Dwellings
1270,E12000001,North East,2004,1129975.05432,Dwellings
1271,E06000005,Darlington UA,2004,45449.2,Dwellings
1272,E06000001,Hartlepool UA,2004,40407.9,Dwellings
1273,E06000002,Middlesbrough UA,2004,58698.8,Dwellings
...,...,...,...,...,...
9724,E10000033,Wiltshire,2023,[x],Dwellings
9725,E07000230,Kennet,2023,[x],Dwellings
9726,E07000231,North Wiltshire,2023,[x],Dwellings
9727,E07000232,Salisbury,2023,[x],Dwellings


In [3]:
long_term_vacants = pd.read_excel(ROOT / 'raw/LT_615.ods', engine='odf', sheet_name='All_long-term_vacants', header=3)
mapper = {}
for col in long_term_vacants.columns.to_list()[2:]:
    mapper[col] = col[6:10]
long_term_vacants.rename(columns=mapper, inplace=True)
dates = long_term_vacants.columns.to_list()[2:]
long_term_vacants = long_term_vacants.melt(id_vars=['CurrentONS code', 'Area'], var_name='date', value_name='Value', value_vars=dates)
long_term_vacants['Measure'] = 'LongTermVacants'
long_term_vacants.rename(columns={'CurrentONS code': 'geography_code', 'Area': 'geography_name'}, inplace=True)
long_term_vacants['geography_name'] = long_term_vacants['geography_name'].str.replace(r"&", 'and', regex=True)
long_term_vacants

,geography_code,geography_name,date,Value,Measure
0,E92000001,England,2004,318642,LongTermVacants
1,E12000001,North East,2004,21599,LongTermVacants
2,E07000157,Alnwick,2004,264,LongTermVacants
3,E07000158,Berwick-upon-Tweed,2004,355,LongTermVacants
4,E07000159,Blyth Valley,2004,277,LongTermVacants
...,...,...,...,...,...
7715,E07000052,West Dorset,2023,[x],LongTermVacants
7716,E07000191,West Somerset,2023,[x],LongTermVacants
7717,E07000233,West Wiltshire,2023,[x],LongTermVacants
7718,E07000053,Weymouth and Portland,2023,[x],LongTermVacants


In [4]:
all_vacants = pd.read_excel(ROOT / 'raw/LT_615.ods', engine='odf', sheet_name='All_vacants', header=3)
mapper = {}
for col in all_vacants.columns.to_list()[2:]:
    mapper[col] = col[6:10]
all_vacants.rename(columns=mapper, inplace=True)
dates = all_vacants.columns.to_list()[2:]
all_vacants = all_vacants.melt(id_vars=['CurrentONS code', 'Area'], var_name='date', value_name='Value', value_vars=dates)
all_vacants['Measure'] = 'AllVacants'
all_vacants.rename(columns={'CurrentONS code': 'geography_code', 'Area': 'geography_name'}, inplace=True)
all_vacants['geography_name'] = all_vacants['geography_name'].str.replace(r"&", 'and', regex=True)
all_vacants

,geography_code,geography_name,date,Value,Measure
0,E92000001,England,2004,710935,AllVacants
1,E12000001,North East,2004,45732,AllVacants
2,E07000157,Alnwick,2004,571,AllVacants
3,E07000158,Berwick-upon-Tweed,2004,614,AllVacants
4,E07000159,Blyth Valley,2004,789,AllVacants
...,...,...,...,...,...
7715,E07000052,West Dorset,2023,[x],AllVacants
7716,E07000191,West Somerset,2023,[x],AllVacants
7717,E07000233,West Wiltshire,2023,[x],AllVacants
7718,E07000053,Weymouth and Portland,2023,[x],AllVacants


In [5]:
projections = pd.read_excel(ROOT / 'raw/2018basedhhpsprincipalprojection.xlsx', sheet_name='406', header=4, engine='openpyxl')
projections = projections.melt(id_vars=['Area code', 'Area name'], var_name='date', value_vars=projections.columns.to_list()[1:], value_name='Value')
projections['Measure'] = 'HouseholdProjection'
projections.rename(columns={'Area code': 'geography_code', 'Area name': 'geography_name'}, inplace=True)
projections = projections.loc[(projections['date'].astype(int) <= 2023) & (projections['date'].astype(int) >= 2004)]
projections

,geography_code,geography_name,date,Value,Measure
1107,E92000001,England,2004,20751955,HouseholdProjection
1108,E12000001,North East,2004,1077847,HouseholdProjection
1109,E06000047,County Durham,2004,209302,HouseholdProjection
1110,E06000005,Darlington,2004,42791,HouseholdProjection
1111,E06000001,Hartlepool,2004,38156,HouseholdProjection
...,...,...,...,...,...
8482,E07000187,Mendip,2023,52019,HouseholdProjection
8483,E07000188,Sedgemoor,2023,55343,HouseholdProjection
8484,E07000189,South Somerset,2023,75729,HouseholdProjection
8485,E07000190,Taunton Deane,2023,54719,HouseholdProjection


In [6]:
combined = pd.concat([dwelling_stock, long_term_vacants, projections, all_vacants])

In [7]:
combined_long = combined.sort_values(by=['geography_code', 'Measure', 'date'])
combined_long = combined_long[['geography_code','geography_name','Measure','date','Value']]
combined_long['date'] = combined_long['date'].astype(int)
combined_long['Value'] = combined_long['Value'].astype(float, errors='ignore').round(0)
combined_long.replace('[x]', pd.NA, inplace=True)
combined_long.dropna(inplace=True)
combined_long['Value'] = combined_long['Value'].round(0).astype(int)

combined_long['geography_name'] = combined_long['geography_name'].str.replace('UA', '')
combined_long['geography_name'] = combined_long['geography_name'].str.replace(' (Met County)', '')
combined_long['geography_name'] = combined_long['geography_name'].str.strip()

combined_long.to_csv(ROOT / 'data/vacant-homes/standard/AllCombined_Cleaned_2024.csv', index=False)

In [8]:
combined_wide = combined_long.pivot(index=["geography_code", "geography_name", "date"], columns="Measure", values="Value")
# combined_wide.to_csv(ROOT / 'data/vacant-homes/absolute.csv')
combined_wide.to_parquet(ROOT / 'data/vacant-homes/site/absolute.parquet')

In [9]:
combined_wide['LongTermVacants_pct'] = (combined_wide['LongTermVacants'] / combined_wide['Dwellings']).mul(100).round(2)
combined_wide['AllVacants_pct'] = (combined_wide['AllVacants'] / combined_wide['Dwellings']).mul(100).round(2)
percentages_wide = combined_wide[['LongTermVacants_pct', 'AllVacants_pct']].dropna()
percentages_wide.to_parquet(ROOT / 'data/vacant-homes/site/percentages.parquet')

In [10]:
# percentages_wide.to_csv(ROOT / 'data/vacant-homes/percentages.csv')


In [11]:
most_recent = percentages_wide[percentages_wide.index.get_level_values('date') == 2023]

In [12]:
ltv = most_recent.copy()
for i, row in most_recent.iterrows():
    ltv.loc[i, 'title'] = 'Long term vacants'
    ltv.loc[i, 'value'] = row['LongTermVacants_pct']
    eng_vals = most_recent[most_recent.index.get_level_values('geography_name')=='England']
    ltv.loc[i, 'footnote'] = f"of homes are long term vacant compared to {eng_vals['LongTermVacants_pct'].values[0]}% for all England"

av = ltv.copy()
for i, row in av.iterrows():
    av.loc[i, 'title'] = 'All vacants'
    av.loc[i, 'value'] = row['AllVacants_pct']
    eng_vals = most_recent[most_recent.index.get_level_values('geography_name')=='England']
    av.loc[i, 'footnote'] = f"of homes are vacant compared to {eng_vals['AllVacants_pct'].values[0]}% for all England"

av.drop(columns=['LongTermVacants_pct', 'AllVacants_pct'], inplace=True)
ltv.drop(columns=['LongTermVacants_pct', 'AllVacants_pct'], inplace=True)
ltv.index = ltv.index.droplevel([1, 2])
av.index = av.index.droplevel([1, 2])
headlines = pd.concat([ltv, av])

In [13]:
headlines.to_parquet(ROOT / 'data/vacant-homes/site/headlines.parquet')